# Implementation of baseline using Artificial Neural Network with Win-loss feature set and SA

Necessary files: ann_functions.py, testing_functions.py, win_loss_functions.py, SA_17.csv, SA_18.csv, SA_19.csv

In [ ]:
import ann_functions
import testing_functions
import win_loss_functions
import pandas as pd
from datetime import datetime
import requests
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tqdm import tqdm

The baseline is implemented according to the paper An Improved Prediction System for Football a Match Result bu C. P. Igiri and E. O. Nwachukwu from 2014. The features based only on results of previous matches are used and the model is evaluated on the SA dataset.

The authors used only one season for training, so I did the same. I used seasons 2016/17 for training season 2017/18 as validation set and the second half of season 2018/19 for testing. The validation set is used to select the best checkpoint of the training according to the validation accuracy.

In [ ]:
X_train, y_train = win_loss_functions.create_data(['SA_17.csv'], skip_rounds = 6)
results_val, matches_per_round = win_loss_functions.create_data_single('SA_18.csv', ['SA_17.csv'], skip_rounds = 6)
# Dates are returned as well for dividing testing season into slices
results_test, matches_per_round = win_loss_functions.create_data_single('SA_19.csv', ['SA_17.csv', 'SA_18.csv'],
                                return_dates=True, skip_rounds = 6)

Processing SA_17.csv season file.


In [ ]:
X_val = results_val.drop('FTR', axis=1)
y_val = results_val['FTR']
# The classes need to go from 0 to 2 not from -1 to 1.
y_train += 1
y_val += 1

They didnt say what ANN architecture they used. I used 3 dense layers with 64, 32 and 16 neurons. I used dropout to overcome overfitting.

I always let the model train for 100 epochs and took the weights from the best epoch according to validation accuracy.

To use all the data available, the first half of the testing season was added to the training data.

In [ ]:
# Some rounds in the beginning are ignored, this is the correct index
# of the start of the second half of the season
start_test_index = 13 * matches_per_round

In [ ]:
X_test_to_append, y_test_to_append = testing_functions.prepare_test_to_append(results_test,
                                                                              start_test_index)
y_test_to_append += 1

For evaluation on the testing dataset I divided the testing data into rounds approximately. The model is trained on the training dataset, then it is evaluated on one round of the testing dataset and this round is added to the training dataset.

In [ ]:
# Adding 1st half of testing season to the training data
X_train = pd.concat([X_train, X_test_to_append])
y_train = pd.concat([y_train, y_test_to_append])
# Rounds of the testing dataset
slices = testing_functions.get_slices(results_test, matches_per_round,
                                      start_test_index)
weighted_sum = 0
sum = 0
for slc in tqdm(slices):
  # Creating the model
  model = ann_functions.func_model((X_train.shape[1],))
  model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
  mc = tf.keras.callbacks.ModelCheckpoint('./weights_model.h5',
                                     monitor='val_accuracy',
                                     save_weights_only=True,
                                     save_best_only=True)
  X_test = slc.drop(['FTR', 'Date'], axis=1)
  y_test = slc['FTR']
  y_test += 1
  # Train the model
  history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=8,
                    validation_data=(X_val, y_val),
                    verbose = 0,
                    callbacks=[mc])
  # Load the best checkpoint
  model.load_weights('weights_model.h5')
  weighted_sum += (model.evaluate(X_test, y_test)[1] * len(y_test))
  sum += len(y_test)
  # Add the round to the training dataset
  X_train = pd.concat([X_train, X_test])
  y_train = pd.concat([y_train, y_test])
print('')
print(weighted_sum / sum)

  0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 0s 141ms/step - loss: 0.9641 - accuracy: 0.4615


  7%|▋         | 1/15 [00:21<04:58, 21.34s/it]

1/1 [==============================] - 0s 126ms/step - loss: 1.2203 - accuracy: 0.2667


 13%|█▎        | 2/15 [00:42<04:37, 21.32s/it]

...

1/1 [==============================] - 0s 137ms/step - loss: 1.0170 - accuracy: 0.4375


 93%|█████████▎| 14/15 [04:50<00:20, 20.64s/it]

1/1 [==============================] - 0s 149ms/step - loss: 1.0475 - accuracy: 0.4545


100%|██████████| 15/15 [05:13<00:00, 20.90s/it]


0.4789473754794974


The testing accuracy is 47.89%.